# YT Comments analysis

In [1]:
from googleapiclient.discovery import build
import pandas as pd
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel

youtube = build('youtube', 'v3', developerKey=API_KEY)

exemple_video = "qCKyRhkhqoQ"

/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:100: SyntaxWarning: invalid escape sequence '\*'
  """Create a SingleValueConstraint object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:141: SyntaxWarning: invalid escape sequence '\*'
  """Create a ContainedSubtypeConstraint object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:301: SyntaxWarning: invalid escape sequence '\*'
  """Create a PermittedAlphabetConstraint object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:457: SyntaxWarning: invalid escape sequence '\*'
  """Create a ConstraintsIntersection logic operator object.
/usr/lib/python3/dist-packages/pyasn1/type/constraint.py:501: SyntaxWarning: invalid escape sequence '\*'
  """Create a ConstraintsUnion logic operator object.
/usr/lib/python3/dist-packages/pyasn1/type/namedval.py:15: SyntaxWarning: invalid escape sequence '\*'
  """Create named values object.
/tmp/ipykernel_253405/2095367673.py:2: DeprecationWarning: 
Pyarrow will bec

In [2]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    data = {
        "channel_name": response['items'][0]['snippet']['title'],
        "channel_id": response['items'][0]['id'],
        "stats": response['items'][0]['statistics']
    }
    return data

In [3]:
def get_video_comments(youtube,video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=1000,
    )
    response = request.execute()
    return response

In [4]:
def get_video_comments_texts(youtube, video_Id):
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_Id,
        maxResults=1000,
    )
    response = request.execute()
    
    # Extract textOriginal from each item
    comments_texts = []
    for item in response.get("items", []):
        if "snippet" in item and "topLevelComment" in item["snippet"]:
            comment_snippet = item["snippet"]["topLevelComment"]["snippet"]
            if "textOriginal" in comment_snippet:
                comments_texts.append(comment_snippet["textOriginal"])
    
    return comments_texts

In [5]:
get_channel_stats(youtube, channel_id)
video = get_video_comments_texts(youtube, exemple_video)

In [8]:
pd.set_option('display.max_rows', False)

df = pd.DataFrame({"Com":video})

df

,Com
0,Je suis en fou rire sur le calendrier
1,Salut à tous les quatre et à toutes la prod vo...
2,Comment ça va grave imprimer le calendrier et ...
3,Zafeel en octobre c'est quelque chose 😭😭
4,On peut encore t’envoyer des colis ?
5,Hello
6,Hello
7,La masterclass des gage c'est trop 😂😂😂
8,Mastu qui est tendu toute les 10sec 😅
...,...


In [7]:
from langdetect import detect, LangDetectException

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

df["Language"] = df["Com"].apply(detect_language)
df

,Com,Language
0,Je suis en fou rire sur le calendrier,fr
1,Salut à tous les quatre et à toutes la prod vo...,fr
2,Comment ça va grave imprimer le calendrier et ...,fr
3,Zafeel en octobre c'est quelque chose 😭😭,fr
4,On peut encore t’envoyer des colis ?,fr
5,Hello,fi
6,Hello,fi
7,La masterclass des gage c'est trop 😂😂😂,fr
8,Mastu qui est tendu toute les 10sec 😅,fr
...,...,...


In [1]:
import nltk

